In [1]:
import torch
import torch.nn as nn
from dataset import LargeMovieDataset
from torch.utils.data import DataLoader
from gensim.models import Word2Vec
import torch.nn.functional as F

#DATALOADER
word_embedding_size = 64
word2vec_model = Word2Vec.load("../checkpoints/word2vec_model.model")

train_set = LargeMovieDataset(path="../data/aclImdb/", set="train", embedding_dic=word2vec_model.wv, word_embedding_size=word_embedding_size)
test_set = LargeMovieDataset(path='../data/aclImdb/', set='test', embedding_dic=word2vec_model.wv, word_embedding_size=word_embedding_size)

train_set, val_set = torch.utils.data.random_split(train_set, [int(0.8 * len(train_set)), len(train_set) - int(0.8 * len(train_set))])

train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=32, shuffle=True)

Using cuda


[nltk_data] Downloading package punkt to /home/mli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Max length of the sentences: 1000
Max length of the sentences: 1000


In [ ]:
class CustomRNN(nn.Module):
    """Hyperparameters : [hiden_size, num_layer]"""
    def __init__(self, input_size, hidden_size, num_layer):
        super(CustomRNN, self).__init__()
        self.input_size = input_size        
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.c1 = nn.RNN(self.input_size, self.hidden_size, num_layers=num_layer, batch_first=True)
        self.c2 = nn.Linear(self.hidden_size, 2)

    def forward(self, x, mask):
        x, hiden = self.c1(x)
        x = x[:, -1, :]
        x = self.c2(x)
        return F.relu(x)
    
class CustomLSTM(nn.Module):
    """Hyperparameters : [hiden_size, num_layer]"""
    def __init__(self, input_size, hidden_size, num_layer, bidirectional_bool):
        super(CustomLSTM, self).__init__()
        self.input_size = input_size        
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.c1 = nn.LSTM(self.input_size, int(self.hidden_size / (bidirectional_bool + 1)), num_layers=num_layer, batch_first=True, bidirectional=bidirectional_bool)
        self.c2 = nn.Linear(self.hidden_size, 2)

    def forward(self, x, mask):
        x, hiden = self.c1(x)
        x = x[:, -1, :]
        x = self.c2(x)
        return F.relu(x)

hidensize = 128
num_layer = 1
rnn_model = CustomRNN(word_embedding_size, hidensize, num_layer)
lstm_model = CustomLSTM(word_embedding_size, hidensize, num_layer, False)
bilstm_model = CustomLSTM(word_embedding_size, hidensize, num_layer, True)

learning_rate=1e-3
optimizer_rnn = torch.optim.Adam(rnn_model.parameters(), lr=learning_rate)
criterion_rnn = nn.CrossEntropyLoss()

In [4]:
from utils import train

train(model=rnn_model, train_dataloader=train_dataloader, val_dataloader=val_dataloader, optimizer=optimizer_rnn, criterion=criterion_rnn, num_epochs=5)
train(model=lstm_model, train_dataloader=train_dataloader, val_dataloader=val_dataloader, optimizer=optimizer_rnn, criterion=criterion_rnn, num_epochs=5)
train(model=bilstm_model, train_dataloader=train_dataloader, val_dataloader=val_dataloader, optimizer=optimizer_rnn, criterion=criterion_rnn, num_epochs=5)

Using device: cuda:0 to train

Epoch: 0/5


100%|██████████| 625/625 [00:13<00:00, 46.27batch/s, acc=25.00%, epoch=0, loss=0.693, set=train]


Train: loss: 0.6931547475814819, accuracy: 50.12515018021626%


100%|██████████| 157/157 [00:02<00:00, 56.71batch/s, acc=66.67%, epoch=0, loss=0.693, set=val]


Validation: loss: 0.6931474785895864, accuracy: 49.589589589589586%
Epoch: 1/5


...

Validation: loss: 0.6931474785895864, accuracy: 49.589589589589586%
Epoch: 4/5


100%|██████████| 625/625 [00:17<00:00, 35.96batch/s, acc=50.00%, epoch=4, loss=0.693, set=train]


Train: loss: 0.6931471017837525, accuracy: 50.12515018021626%


100%|██████████| 157/157 [00:03<00:00, 51.86batch/s, acc=33.33%, epoch=4, loss=0.693, set=val]


Validation: loss: 0.6931474785895864, accuracy: 49.589589589589586%
